In [1]:

import os
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import json 
import re
from typing import Optional
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
from plotly.subplots import make_subplots
import pandas as pd
import re
import math

# Using batch sizes from grafana

In [ ]:
def find_uuid(file_name: str) -> Optional[str]:
    match = re.search(r'[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}', file_name)
    uuid = None
    if match:
        uuid = match.group()
    else:
        raise ValueError(f"UUID not found in filename {file_name}")
        
    return uuid


def read_json_files_to_df(directory: str) -> pd.DataFrame:
    data = []

    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('individual_responses.json'):
            model_name = '_'.join(filename.split('_')[2:3])
            file_path = os.path.join(directory, filename)
            # Read the JSON file
            with open(file_path, 'r') as file:
                json_data = json.load(file)
                # Extract relevant fields from each JSON object and append to the data list
                for item in json_data:
                    if pd.isnull(item['error_code']):
                        data.append(
                            {
                                'start_time': datetime.strptime(item['start_time'], "%H:%M:%S.%f"),
                                'end_time': datetime.strptime(item['end_time'], "%H:%M:%S.%f"),
                                'client_end_to_end_latency_s': item['client_end_to_end_latency_s'],
                                'client_ttft_s': item['client_ttft_s'],
                                'model_name': model_name,
                                'uuid': find_uuid(filename),
                            }
                        )
    # Create a DataFrame from the data list
    df = pd.DataFrame(data)
    return df

def rgb_to_hex(rgb_str: str) -> str:
    """Convert 'rgb(r,g,b)' string to hex string like '#rrggbb'."""
    nums = list(map(int, re.findall(r"\d+", rgb_str)))
    return "#{:02x}{:02x}{:02x}".format(*nums)

def lighten_color(color: str, factor: float = 0.6) -> str:
    """
    Lightens a color (hex or rgb string) by interpolating toward white.
    Always returns a hex string.
    """
    if color.startswith("rgb"):
        color = rgb_to_hex(color)
    r, g, b = pc.hex_to_rgb(color)
    lighter_rgb = (
        int(r + (255 - r) * factor),
        int(g + (255 - g) * factor),
        int(b + (255 - b) * factor),
    )
    return "#{:02x}{:02x}{:02x}".format(*lighter_rgb)

def plot_requests_gantt_chart(df_user: pd.DataFrame, output_dir: str, file_name: str):
    """
    Plots a Gantt chart of response timings across all requests,
    with per-model metric colors and layered TTFT vs latency.
    """    
    requests = df_user.index + 1
    fig = go.Figure()

    # Assign a base color per model
    unique_models = df_user["model_name"].unique()
    palette = pc.qualitative.Set2
    color_map = {model: palette[i % len(palette)] for i, model in enumerate(unique_models)}

    # Add bars for each metric and model
    for model in unique_models:
        model_df = df_user[df_user["model_name"] == model]

        # End-to-end latency → lighter color
        fig.add_trace(
            go.Bar(
                y=model_df.index + 1,
                x=1000 * model_df["client_end_to_end_latency_s"],
                base=[str(x) for x in model_df["start_time"]],
                name=f"End-to-end latency - {model}",
                orientation="h",
                marker_color=lighten_color(color_map[model], factor=0.5),
                legendgroup=model,
                offsetgroup=f"{model}-latency",
            )
        )

        # TTFT → normal color (plotted after to be on top)
        fig.add_trace(
            go.Bar(
                y=model_df.index + 1,
                x=1000 * model_df["client_ttft_s"],
                base=[str(x) for x in model_df["start_time"]],
                name=f"TTFT - {model}",
                orientation="h",
                marker_color=lighten_color(color_map[model], factor=0.0),  # keep base color
                legendgroup=model,
                offsetgroup=f"{model}-ttft",
            )
        )

    # Alternate row shading
    for i in range(0, len(df_user.index), 2):
        fig.add_hrect(y0=i + 0.5, y1=i + 1.5, line_width=0, fillcolor="grey", opacity=0.1)

    fig.update_xaxes(
        type="date",
        tickformat="%H:%M:%S",
        hoverformat="%H:%M:%S.%2f",
    )
    fig.update_layout(
        barmode="overlay",  # ensures TTFT overlays latency
        title_text="LLM requests across time",
        xaxis_title="Time stamp",
        yaxis_title="Request index",
    )

    fig.show()
    fig.write_html(f"{output_dir}/llms_across_time-{file_name}.html", include_plotlyjs="cdn")

    return fig

## 2_ckpts-100_conc_reqs-50_50

In [9]:
df_test1 = read_json_files_to_df('../data/bundle_tests/amit_tests_v2/2_ckpts-100_conc_reqs-50_50/20250915-180848.094978')
df_test1 = df_test1.sort_values('end_time').reset_index(drop=True)

In [11]:
plot_requests_gantt_chart(df_test1, file_name="test1")